In [1]:
import torch
print("Checking runtime configuration...")
if torch.cuda.is_available():
    print(f"GPU Runtime: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("GPU NOT AVAILABLE!")
    print("To fix: Runtime -> Change runtime type -> Hardware accelerator -> GPU")
import sys
print(f"Python Version: {sys.version}")
print("Runtime check complete!")

Checking runtime configuration...
GPU Runtime: Tesla T4
CUDA Version: 12.6
GPU Memory: 15.8 GB
Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Runtime check complete!


In [2]:
import os
os.chdir('/content')
print("Installing system dependencies...")
!apt-get update -qq
!apt-get install -y -qq ffmpeg libsm6 libxext6 libxrender-dev libglib2.0-0
print("Installing Python packages...")
!pip install -q "numpy>=2.0.0,<3.0.0"
!pip install -q "protobuf>=5.26.1,<7.0.0"
!pip install -q pyngrok
print("Dependencies installed!")

Installing system dependencies...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Installing Python packages...
Dependencies installed!


In [3]:
import shutil
# UPDATE THIS URL
REPO_URL = "https://github.com/bitcoin4cashqc/Deep-Live-Cam-Server.git"
BRANCH = "Server"
print(f"Cloning repository: {REPO_URL}")
print(f"Branch: {BRANCH}")
project_dir = "/content/Deep-Live-Cam-Server"
if os.path.exists(project_dir):
    shutil.rmtree(project_dir)
os.chdir('/content')
!git clone -b {BRANCH} {REPO_URL}
os.chdir(project_dir)
print(f"Repository cloned to: {os.getcwd()}")
!git branch --show-current
key_files = ['server_ws.py', 'client_ws.py', 'requirements.txt', 'modules/websocket_server.py']
for file in key_files:
    if os.path.exists(file):
        print(f"Found: {file}")
    else:
        print(f"Missing: {file}")

Cloning repository: https://github.com/bitcoin4cashqc/Deep-Live-Cam-Server.git
Branch: Server
Cloning into 'Deep-Live-Cam-Server'...
remote: Enumerating objects: 1563, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 1563 (delta 25), reused 20 (delta 10), pack-reused 1519 (from 2)
Receiving objects: 100% (1563/1563), 152.12 MiB | 20.50 MiB/s, done.
Resolving deltas: 100% (888/888), done.
Repository cloned to: /content/Deep-Live-Cam-Server
Server
Found: server_ws.py
Found: client_ws.py
Found: requirements.txt
Found: modules/websocket_server.py


In [4]:
print("Downloading required models...")
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# Download inswapper model
print("Downloading inswapper_128_fp16.onnx...")
!wget -q -O models/inswapper_128_fp16.onnx "https://huggingface.co/hacksider/deep-live-cam/resolve/main/inswapper_128_fp16.onnx?download=true"

# Download GFPGAN model
print("Downloading GFPGANv1.4.pth...")
!wget -q -O models/GFPGANv1.4.pth "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth"

print("Models downloaded successfully!")
print(f"Models saved in: {os.path.join(os.getcwd(), models_dir)}")

# Verify downloads
model_files = ["inswapper_128_fp16.onnx", "GFPGANv1.4.pth"]
for model_file in model_files:
    model_path = os.path.join(models_dir, model_file)
    if os.path.exists(model_path):
        size_mb = os.path.getsize(model_path) / (1024 * 1024)
        print(f"✓ {model_file}: {size_mb:.1f} MB")
    else:
        print(f"✗ {model_file}: NOT FOUND")

Models downloaded successfully!
Models saved in: /content/Deep-Live-Cam-Server/models
✓ inswapper_128_fp16.onnx: 264.4 MB
✓ GFPGANv1.4.pth: 332.5 MB


In [5]:
print("Installing project dependencies...")
if torch.cuda.is_available():
    print("Installing PyTorch with CUDA...")
    !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    !pip install -q onnxruntime-gpu
else:
    print("Installing PyTorch CPU...")
    !pip install -q torch torchvision torchaudio
    !pip install -q onnxruntime
!pip install -q opencv-python==4.10.0.84
!pip install -q "websockets>=11.0"
!pip install -q insightface==0.7.3
!pip install -q "pillow>=11.0.0"
!pip install -q psutil
!pip install -q opennsfw2
!pip install -q "git+https://github.com/xinntao/BasicSR.git@master"
!pip install -q "git+https://github.com/TencentARC/GFPGAN.git@master"
import torch
if torch.cuda.is_available():
    print(f"GPU ready: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")
print("Dependencies installed!")

Installing project dependencies...
Installing PyTorch with CUDA...
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
GPU ready: Tesla T4
Dependencies installed!


In [6]:
from pyngrok import ngrok, conf
# ADD YOUR TOKEN HERE
NGROK_TOKEN = "6oL7zUZ6WPjQoTCnar3Yt_7ATyJfwNHKZw4Kh9uMZNA"
if NGROK_TOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("Add your ngrok token above!")
    print("Get it from: https://dashboard.ngrok.com/get-started/your-authtoken")
    websocket_url = "ws://localhost:8765"
    print(f"Using localhost: {websocket_url}")
else:
    conf.get_default().auth_token = NGROK_TOKEN
    ngrok.kill()
    try:
        tunnel = ngrok.connect(8765, "http")
        tunnel_url = tunnel.public_url
        websocket_url = f"ws://{tunnel_url.replace('http://', '')}"
        print(f"Ngrok tunnel created: {tunnel_url}")
        print(f"WebSocket URL: {websocket_url}")
        print(f"Use this in clients: python client_ws.py -s face.jpg --server-url {websocket_url}")
    except Exception as e:
        print(f"Ngrok failed: {e}")
        websocket_url = "ws://localhost:8765"
        print(f"Using localhost: {websocket_url}")

Ngrok tunnel created: tcp://6.tcp.ngrok.io:16814
WebSocket URL: ws://6.tcp.ngrok.io:16814
Use this in clients: python client_ws.py -s face.jpg --server-url ws://6.tcp.ngrok.io:16814


In [7]:

import subprocess
import sys
print("Starting Deep Live Cam WebSocket Server...")
print(f"WebSocket URL: {websocket_url}")
print(f"GPU: {'Available' if torch.cuda.is_available() else 'CPU Only'}")
print(f"Directory: {os.getcwd()}")
print("Client examples:")
print(f"  python client_ws.py -s alice.jpg --server-url {websocket_url}")
print(f"  python client_ws.py -s bob.jpg --server-url {websocket_url}")
cmd = [sys.executable, "server_ws.py", "--headless"]
if torch.cuda.is_available():
    cmd.extend(["--execution-provider", "cuda", "--execution-threads", "4"])
    print("Using CUDA")
else:
    cmd.extend(["--execution-provider", "cpu", "--execution-threads", "2"])
    print("Using CPU")
cmd.extend(["--server-port", "8765"])
print(f"Starting: {' '.join(cmd)}")
print("-" * 50)
try:
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True, bufsize=1)
    print("Server started! Press Interrupt to stop.")
    for line in process.stdout:
        print(line.strip())
except KeyboardInterrupt:
    print("Server stopped by user")
    process.terminate()
except Exception as e:
    print(f"Server error: {e}")
finally:
    try:
        ngrok.kill()
    except:
        pass

Starting Deep Live Cam WebSocket Server...
WebSocket URL: ws://6.tcp.ngrok.io:16814
GPU: Available
Directory: /content/Deep-Live-Cam-Server
Client examples:
  python client_ws.py -s alice.jpg --server-url ws://6.tcp.ngrok.io:16814
  python client_ws.py -s bob.jpg --server-url ws://6.tcp.ngrok.io:16814
Using CUDA
Starting: /usr/bin/python3 server_ws.py --headless --execution-provider cuda --execution-threads 4 --server-port 8765
--------------------------------------------------
Server started! Press Interrupt to stop.
2025-10-15 06:36:50.311431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760510210.345531    9367 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760510210.356738    9367 cuda_blas.cc:1407] Unable to register cuBLAS factory:

In [8]:
import asyncio
import websockets
import json
async def test_connection():
    try:
        ws = await websockets.connect("ws://localhost:8765")
        await ws.send(json.dumps({'type': 'stats_request'}))
        response = await asyncio.wait_for(ws.recv(), timeout=5)
        data = json.loads(response)
        await ws.close()
        if data.get('type') == 'stats':
            stats = data.get('data', {})
            print(f"Server running! Clients: {stats.get('connected_clients', 0)}")
            return True
        return False
    except Exception as e:
        print(f"Test failed: {e}")
        return False
result = await test_connection()
if result:
    print(f"Server ready at: {websocket_url}")
else:
    print("Server not responding")

Test failed: did not receive a valid HTTP response
Server not responding
